# **LangChain Memory**

LangChain provides a few different options for storing  a conversation memory:
- Storing all messages
- Storing a limited number of interactions
- Storing a limited number of tokens

## **What's covered?**
- ChatMessageHistory
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationSummaryMemory

## **ChatMessageHistory**

This allows us to mannualy add the messages to memory.

In [1]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

In [2]:
history.add_user_message("Hello, Nice to meet you.")

In [3]:
history.add_ai_message("Nice to meet you!")

In [4]:
history.messages

[HumanMessage(content='Hello, Nice to meet you.'),
 AIMessage(content='Nice to meet you!')]

## **ConversationBufferMemory**

This memory type can be connected to a conversation. It allows for storing messages and then extracts the messages in a variable.

In [5]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [6]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [7]:
# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Set up an instance of Conversation buffer memory
memory = ConversationBufferMemory()

In [8]:
# Set up an assistant
conversation = ConversationChain(llm=chat_model, memory=memory, verbose=True)

In [9]:
conversation.predict(input="Hello, Nice to meet you.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, Nice to meet you.
AI:

> Finished chain.


"Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?"

In [10]:
conversation.predict(input="Tell me a nice machine learning fact.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, Nice to meet you.
AI: Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?
Human: Tell me a nice machine learning fact.
AI:

> Finished chain.


"Sure! Did you know that machine learning algorithms can analyze large amounts of data to find patterns and make predictions without being explicitly programmed to do so? It's pretty fascinating how they can learn and improve over time based on the data they are given."

In [11]:
print(memory.buffer)

Human: Hello, Nice to meet you.
AI: Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?
Human: Tell me a nice machine learning fact.
AI: Sure! Did you know that machine learning algorithms can analyze large amounts of data to find patterns and make predictions without being explicitly programmed to do so? It's pretty fascinating how they can learn and improve over time based on the data they are given.


In [14]:
print(conversation.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, Nice to meet you.'), AIMessage(content="Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?"), HumanMessage(content='Tell me a nice machine learning fact.'), AIMessage(content="Sure! Did you know that machine learning algorithms can analyze large amounts of data to find patterns and make predictions without being explicitly programmed to do so? It's pretty fascinating how they can learn and improve over time based on the data they are given.")])


**Let's now learn to save this history on the disk so that whenever we can load the history whenever we chat with our assistant.**

In [17]:
import pickle

chat_history = pickle.dumps(conversation.memory)

with open("chats_data/conversation_memory.pkl", "wb") as f:
    f.write(chat_history)

## **Loading a Chat History**

In [20]:
chat_history_loaded = pickle.load(open("chats_data/conversation_memory.pkl", "rb"))

print(chat_history_loaded)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, Nice to meet you.'), AIMessage(content="Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?"), HumanMessage(content='Tell me a nice machine learning fact.'), AIMessage(content="Sure! Did you know that machine learning algorithms can analyze large amounts of data to find patterns and make predictions without being explicitly programmed to do so? It's pretty fascinating how they can learn and improve over time based on the data they are given.")])


In [21]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

In [22]:
# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Set up an instance of Conversation buffer memory
memory_reloaded = chat_history_loaded

conversation_reloaded = ConversationChain(llm=chat_model, memory=memory_reloaded, verbose=True)

In [23]:
print(memory_reloaded.buffer)

Human: Hello, Nice to meet you.
AI: Hello! It's nice to meet you too. I am an AI programmed to assist you with any questions or tasks you may have. How can I help you today?
Human: Tell me a nice machine learning fact.
AI: Sure! Did you know that machine learning algorithms can analyze large amounts of data to find patterns and make predictions without being explicitly programmed to do so? It's pretty fascinating how they can learn and improve over time based on the data they are given.


## **ConversationBufferWindowMemory**

With this we will have an ability to call back just a window of conversation.

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [25]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

In [27]:
# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

# Set up an instance of Conversation buffer window memory
window_memory = ConversationBufferWindowMemory(k=1)
# here, k most recent interactions will be refered to answer the user queries

# Set up an assistant
conversation = ConversationChain(llm=chat_model, memory=window_memory, verbose=True)

In [29]:
conversation.predict(input="Hello, How are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, How are you?
AI:

> Finished chain.


"Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?"

In [30]:
conversation.predict(input="What is the new thing you learned recently?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, How are you?
AI: Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?
Human: What is the new thing you learned recently?
AI:

> Finished chain.


"I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks."

In [31]:
conversation.predict(input="Sounds interesting. Can you teach it to me in a simple term?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is the new thing you learned recently?
AI: I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks.
Human: Sounds interesting. Can you teach it to me in a simple term?
AI:

> Finished chain.


'Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on key details.'

In [33]:
print(conversation.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, How are you?'), AIMessage(content="Hello! I'm doing well, thank you for asking. I've been busy processing information and learning new things. How about you?"), HumanMessage(content='What is the new thing you learned recently?'), AIMessage(content="I recently learned about a new algorithm for image recognition that improves accuracy by 10%. It's called Convolutional Neural Networks with Attention Mechanism (CNN-AM). This algorithm focuses on specific parts of an image to make more accurate predictions. I found it fascinating how it can significantly improve image recognition tasks."), HumanMessage(content='Sounds interesting. Can you teach it to me in a simple term?'), AIMessage(content='Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on k

In [32]:
print(window_memory.buffer)

Human: Sounds interesting. Can you teach it to me in a simple term?
AI: Sure! In simple terms, the CNN-AM algorithm is like a detective that pays extra attention to important parts of a picture to figure out what the picture is showing. It helps computers recognize images more accurately by focusing on key details.


## **ConversationSummaryBufferMemory**

It allows us to call a LLM for a summary of all the historical conversation.

`ConversationSummaryBufferMemory` combines the two ideas. It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both. It uses token length rather than number of interactions to determine when to flush interactions.

In [37]:
from langchain.memory import ConversationSummaryBufferMemory

summary_memory = ConversationSummaryBufferMemory(llm=chat_model, max_token_limit=100)

conversation = ConversationChain(llm=chat_model, memory=summary_memory)

In [38]:
conversation.predict(input="Give me a study plan for Python Programming for my upcoming Data Science Interview.")

"Sure! For your upcoming Data Science interview, I recommend starting with the basics of Python programming. Here's a suggested study plan:\n\n1. Week 1: Introduction to Python - Learn about Python syntax, data types, variables, and basic operations.\n2. Week 2: Control flow and functions - Dive into if statements, loops, and functions in Python.\n3. Week 3: Data structures - Explore lists, dictionaries, tuples, and sets in Python.\n4. Week 4: File handling and modules - Understand how to read and write files, as well as import modules in Python.\n5. Week 5: Object-oriented programming - Learn about classes, objects, and inheritance in Python.\n6. Week 6: Libraries and frameworks - Familiarize yourself with popular libraries like NumPy, Pandas, and Matplotlib for data manipulation and visualization.\n7. Week 7: Practice coding exercises - Solve coding challenges on platforms like LeetCode or HackerRank to reinforce your skills.\n8. Week 8: Review and mock interviews - Review key concep

In [39]:
conversation.predict(input="Also share for Machine Learning.")

"For Machine Learning, I recommend a 12-week study plan to cover the fundamentals of machine learning algorithms, data preprocessing, model evaluation, feature selection, and tuning hyperparameters. I also suggest diving into specific topics such as regression, classification, clustering, and neural networks. Additionally, it's important to work on real-world projects and participate in Kaggle competitions to apply your knowledge and showcase your skills to potential employers during the interview process. Let me know if you need more specific details or resources for your study plan!"

In [40]:
print(summary_memory.buffer)

[]


In [41]:
print(conversation.memory)

llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000024EA67AC2B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000024EA67AF7C0>, openai_api_key=SecretStr('**********'), openai_proxy='') max_token_limit=100 moving_summary_buffer='The human asks the AI for study plans for Python Programming and Machine Learning for an upcoming Data Science Interview. The AI provides a detailed 8-week study plan for Python and a 12-week study plan for Machine Learning, emphasizing the importance of regular practice, real-world application, and participating in Kaggle competitions to showcase skills.'
